# State Farm Distracted Driver Detection - VGG16 Continued

This notebook contains the final attempt at the Kaggle State Farm Distracted Driver Detection competition using the VGG model. The purpose is to train the best possible base VGG model, as well as testing the external vgg16, utils and plot libraries.

## Initial Setup

Import libraries and functions for future use.

In [1]:
# Plots displayed inline in notebook
%matplotlib inline

# Make help libraries available
import sys

#sys.path.append('/home/ubuntu/personal-libraries')
sys.path.append('D:/anlaursen/libraries')

# Set visible devices, so as to just use a single GPU.
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import numpy as np
import pandas as pd
import gc

from kerastools.vgg16 import Vgg16
from kerastools.utils import get_batches, save_array, load_array, get_classes, do_clip

from keras.models import Model, Sequential
from keras.layers import Input, Dense, BatchNormalization, Flatten, Dropout
from keras.layers.convolutional import MaxPooling2D
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing import image

Using TensorFlow backend.


## Define model

We setup our initial VGG16 model

In [3]:
vgg = Vgg16()
vgg.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
norm_layer (Lambda)          (None, 224, 224, 3)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 226, 226, 3)       0         
_________________________________________________________________
conv_layer_1_0 (Conv2D)      (None, 224, 224, 64)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 226, 226, 64)      0         
_________________________________________________________________
conv_layer_1_1 (Conv2D)      (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
__________

## Setup batches

We define out validation and training badges for modelling

In [4]:
batch_size = 32

#path = ''
path = 'sample/'

train_batches = vgg.get_batches(path + 'train', batch_size = batch_size)
val_batches = vgg.get_batches(path + 'valid', batch_size = batch_size, shuffle = False)

Found 1000 images belonging to 10 classes.
Found 100 images belonging to 10 classes.


## Finetune model - Sample

We need to adjust the standard VGG model to our new input with 10 classes, so we finetune it.

In [5]:
vgg.finetune(train_batches)
vgg.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
norm_layer (Lambda)          (None, 224, 224, 3)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 226, 226, 3)       0         
_________________________________________________________________
conv_layer_1_0 (Conv2D)      (None, 224, 224, 64)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 226, 226, 64)      0         
_________________________________________________________________
conv_layer_1_1 (Conv2D)      (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
__________

We train the model using the default learning rate of 0.001 for a single epoch

In [6]:
vgg.fit_batch(train_batches, val_batches, 1)

Epoch 1/1
31/31 [==============================] - 13s - loss: 4.0784 - acc: 0.1542 - val_loss: 2.0154 - val_acc: 0.2812


We see that the accuracy increases fine on the sample, so we increase the learning rate.

In [7]:
vgg.model.optimizer.lr = 0.01

vgg.fit_batch(train_batches, val_batches, 4)

Epoch 1/4
31/31 [==============================] - 8s - loss: 2.6065 - acc: 0.3216 - val_loss: 1.6577 - val_acc: 0.4706
Epoch 2/4
31/31 [==============================] - 6s - loss: 2.2397 - acc: 0.4044 - val_loss: 2.1991 - val_acc: 0.2647
Epoch 3/4
31/31 [==============================] - 7s - loss: 1.7067 - acc: 0.5036 - val_loss: 1.9854 - val_acc: 0.3824
Epoch 4/4
31/31 [==============================] - 7s - loss: 1.5527 - acc: 0.5371 - val_loss: 1.7916 - val_acc: 0.3529


It actually seems to be generalising nicely. We try 4 more epochs.

In [8]:
vgg.fit_batch(train_batches, val_batches, 4)

Epoch 1/4
31/31 [==============================] - 9s - loss: 1.4975 - acc: 0.5584 - val_loss: 1.9561 - val_acc: 0.3676
Epoch 2/4
31/31 [==============================] - 8s - loss: 1.2034 - acc: 0.6431 - val_loss: 1.4141 - val_acc: 0.5294
Epoch 3/4
31/31 [==============================] - 8s - loss: 1.2958 - acc: 0.6032 - val_loss: 1.7189 - val_acc: 0.4412
Epoch 4/4
31/31 [==============================] - 8s - loss: 1.0274 - acc: 0.6802 - val_loss: 1.5581 - val_acc: 0.5147


Damn. Our valuation accuracy is destroyed. Try 4 more epochs with lower learning rate.

In [9]:
vgg.model.optimizer.lr = 0.001

vgg.fit_batch(train_batches, val_batches, 4)

Epoch 1/4
31/31 [==============================] - 9s - loss: 1.0726 - acc: 0.6655 - val_loss: 1.6395 - val_acc: 0.4853
Epoch 2/4
31/31 [==============================] - 8s - loss: 1.0848 - acc: 0.6654 - val_loss: 1.7437 - val_acc: 0.4559
Epoch 3/4
31/31 [==============================] - 8s - loss: 0.9150 - acc: 0.6996 - val_loss: 1.5928 - val_acc: 0.4706
Epoch 4/4
31/31 [==============================] - 7s - loss: 0.9548 - acc: 0.7007 - val_loss: 1.6661 - val_acc: 0.5147


Seems, this is as far as we can get on the sample data set. A pretty good base line in the area of 0.5

## Finetune model - Full data

We continue our finetuning on the full data set.

In [10]:
path = ''

train_batches = vgg.get_batches(path + 'train', batch_size = batch_size)
val_batches = vgg.get_batches(path + 'valid', batch_size = batch_size, shuffle = False)

Found 19624 images belonging to 10 classes.
Found 2800 images belonging to 10 classes.


We start with a single epoch

In [11]:
vgg.fit_batch(train_batches, val_batches, 1)

Epoch 1/1
613/613 [==============================] - 210s - loss: 1.0573 - acc: 0.6918 - val_loss: 1.4915 - val_acc: 0.5629


We increase the learning rate and see, where that takes us.

In [12]:
vgg.model.optimizer.lr = 0.1

vgg.fit_batch(train_batches, val_batches, 4)

Epoch 1/4
613/613 [==============================] - 168s - loss: 0.8641 - acc: 0.7518 - val_loss: 1.5046 - val_acc: 0.5916
Epoch 2/4
613/613 [==============================] - 178s - loss: 0.8482 - acc: 0.7637 - val_loss: 1.7478 - val_acc: 0.5766
Epoch 3/4
613/613 [==============================] - 162s - loss: 0.8170 - acc: 0.7781 - val_loss: 2.1161 - val_acc: 0.4888
Epoch 4/4
613/613 [==============================] - 159s - loss: 0.8262 - acc: 0.7836 - val_loss: 1.9024 - val_acc: 0.5173


And then we lower the learning rate again, and see where we end up.

In [13]:
vgg.model.optimizer.lr = 0.001

vgg.fit_batch(train_batches, val_batches, 4)

Epoch 1/4
613/613 [==============================] - 159s - loss: 0.8239 - acc: 0.7890 - val_loss: 2.3340 - val_acc: 0.5459
Epoch 2/4
613/613 [==============================] - 158s - loss: 0.8324 - acc: 0.7922 - val_loss: 1.8922 - val_acc: 0.5589
Epoch 3/4
613/613 [==============================] - 159s - loss: 0.8482 - acc: 0.7956 - val_loss: 2.2846 - val_acc: 0.4899
Epoch 4/4
613/613 [==============================] - 158s - loss: 0.8834 - acc: 0.7907 - val_loss: 2.3483 - val_acc: 0.4776


The valuation accuracy is very fluctuating, and generally we are overfitting. Lets try and make more layers trainable and see, if that helps things along.

In [14]:
layers = vgg.model.layers
# Get the index of the first dense layer...
first_dense_idx = [index for index, layer in enumerate(layers) if type(layer) is Dense][0]
# ...and set this and all subsequent layers to trainable
for layer in layers[first_dense_idx:]: layer.trainable = True

And then we rerun the training. First one epoch with low learning rate.

In [15]:
vgg.fit_batch(train_batches, val_batches, 1)

Epoch 1/1
613/613 [==============================] - 162s - loss: 0.8565 - acc: 0.7977 - val_loss: 2.3657 - val_acc: 0.4650


Then four epochs with a higher learning rate.

In [16]:
vgg.model.optimizer.lr = 0.1

vgg.fit_batch(train_batches, val_batches, 4)

Epoch 1/4
613/613 [==============================] - 162s - loss: 0.8592 - acc: 0.8017 - val_loss: 2.6086 - val_acc: 0.4780
Epoch 2/4
613/613 [==============================] - 161s - loss: 0.8587 - acc: 0.8018 - val_loss: 2.2977 - val_acc: 0.5184
Epoch 3/4
613/613 [==============================] - 160s - loss: 0.8650 - acc: 0.8109 - val_loss: 2.4273 - val_acc: 0.5210
Epoch 4/4
613/613 [==============================] - 161s - loss: 0.8732 - acc: 0.8079 - val_loss: 2.1531 - val_acc: 0.5455


And then four epochs with a lower learning rate again.

In [17]:
vgg.model.optimizer.lr = 0.001

vgg.fit_batch(train_batches, val_batches, 4)

Epoch 1/4
613/613 [==============================] - 159s - loss: 0.8897 - acc: 0.8087 - val_loss: 2.4408 - val_acc: 0.5007
Epoch 2/4
613/613 [==============================] - 158s - loss: 0.8835 - acc: 0.8093 - val_loss: 2.3475 - val_acc: 0.5242
Epoch 3/4
613/613 [==============================] - 157s - loss: 0.8668 - acc: 0.8116 - val_loss: 2.3265 - val_acc: 0.5419
Epoch 4/4
613/613 [==============================] - 158s - loss: 0.9162 - acc: 0.8090 - val_loss: 2.2137 - val_acc: 0.5611


That did little to improve things. Let's try an even lower learning rate.

In [18]:
vgg.model.optimizer.lr = 0.00001

vgg.fit_batch(train_batches, val_batches, 4)

Epoch 1/4
613/613 [==============================] - 162s - loss: 0.9038 - acc: 0.8140 - val_loss: 2.4277 - val_acc: 0.5260
Epoch 2/4
613/613 [==============================] - 160s - loss: 0.9479 - acc: 0.8080 - val_loss: 2.6530 - val_acc: 0.5126
Epoch 3/4
613/613 [==============================] - 159s - loss: 0.9349 - acc: 0.8123 - val_loss: 2.2536 - val_acc: 0.5430
Epoch 4/4
613/613 [==============================] - 159s - loss: 0.9275 - acc: 0.8113 - val_loss: 2.2569 - val_acc: 0.5578


We're not improving at all. Let's save the weights and try a different approach.

In [19]:
vgg.model.save_weights('models/base_vgg16.h5')

## Improved VGG

We continue using the VGG16 network, but attempt to improve it. That is we want to keep the pretrained convolutional layers fixed, but use a new architecture for the dense layers.

We start by defining a new VGG16() model.

In [20]:
vgg = Vgg16()
vgg.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
norm_layer (Lambda)          (None, 224, 224, 3)       0         
_________________________________________________________________
zero_padding2d_14 (ZeroPaddi (None, 226, 226, 3)       0         
_________________________________________________________________
conv_layer_1_0 (Conv2D)      (None, 224, 224, 64)      1792      
_________________________________________________________________
zero_padding2d_15 (ZeroPaddi (None, 226, 226, 64)      0         
_________________________________________________________________
conv_layer_1_1 (Conv2D)      (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 112, 112, 64)      0         
__________

We then proceed to find the last max pooling layer of the model.

In [21]:
# Define convolutional layers
last_conv_idx = [i for i, l in enumerate(vgg.model.layers) if type(l) is MaxPooling2D][-1]
conv_layers = vgg.model.layers[:last_conv_idx + 1]

We can then define a model using only the convolutional layers.

In [22]:
conv_model = Sequential(conv_layers)
conv_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
norm_layer (Lambda)          (None, 224, 224, 3)       0         
_________________________________________________________________
zero_padding2d_14 (ZeroPaddi (None, 226, 226, 3)       0         
_________________________________________________________________
conv_layer_1_0 (Conv2D)      (None, 224, 224, 64)      1792      
_________________________________________________________________
zero_padding2d_15 (ZeroPaddi (None, 226, 226, 64)      0         
_________________________________________________________________
conv_layer_1_1 (Conv2D)      (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 112, 112, 64)      0         
__________

The idea is now, that we want to pre-computer all of our data through the convolutional layers. This will drastically reduce the training time, once we start experimenting with dense model architecture.

We start by defining our batches.

In [23]:
path = ''

train_batches = get_batches(path + 'train', batch_size = 44, target_size = (224, 224), shuffle = False)

valid_batches = get_batches(path + 'valid', batch_size = 50, target_size = (224, 224), shuffle = False)

test_batches = get_batches(path + 'test', batch_size = 2, target_size = (224, 224), shuffle = False, class_mode = None)

Found 19624 images belonging to 10 classes.
Found 2800 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


We also extract labels and classes for each dataset.

In [24]:
(val_classes, trn_classes, val_labels, trn_labels, 
 val_filenames, filenames, test_filenames) = get_classes(path)

Found 19624 images belonging to 10 classes.
Found 2800 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


We then pre-compute each of our datasets and save the numpy arrays. This eats a lot of memory on the poor AWS instance, so after each data computation, we do some cleanup to realease the memory. We save and load using bcolz, as it utilises great compression and does I/O very fast.

In [25]:
conv_feat = conv_model.predict_generator(train_batches, np.int(train_batches.samples / train_batches.batch_size))
save_array(path + 'results/conv_computed/conv_feat.dat', conv_feat)

del conv_feat
gc.collect()

5

In [26]:
conv_val_feat = conv_model.predict_generator(valid_batches, np.int(valid_batches.samples / valid_batches.batch_size))
save_array(path + 'results/conv_computed/conv_val_feat.dat', conv_val_feat)

del conv_val_feat
gc.collect()

0

In [27]:
conv_test_feat = conv_model.predict_generator(test_batches, np.int(test_batches.samples / test_batches.batch_size))
save_array(path + 'results/conv_computed/conv_test_feat.dat', conv_test_feat)

del conv_test_feat
gc.collect()

0

And finally, we can load the three feature sets

In [28]:
conv_feat = load_array('results/conv_computed/conv_feat.dat')
conv_val_feat = load_array('results/conv_computed/conv_val_feat.dat')
conv_test_feat = load_array('results/conv_computed/conv_test_feat.dat')

## VGG model with batchnorm dense laysers on pretrained conv layers

Since we've pre-computed the output of the last convolutional layer, we need to create a network that takes that as input, and predicts our 10 classes. Let's try using a simplified version of VGG's dense layers.

In [29]:
def batch_norm_model(base_model, p):
    """
    Function providing a more modern dense layer to the VGG16.
    I.e. it utilizes batch normalisation.
    
    Args:
        - base_model: A keras model object.
        - p: Total level of dropout (will be devided by two in first layers)
        
    Returns:
        - A keras model object ready to be compiled.
    """
    
    inputs = Input(shape = base_model[-1].output_shape[1:])
        
    flat_layer = Flatten()(inputs)
        
    dense_layer_1 = Dropout(p / 2)(flat_layer)
    dense_layer_1 = Dense(128, activation='relu')(dense_layer_1)
    dense_layer_2 = BatchNormalization()(dense_layer_1)
    dense_layer_2 = Dropout(p / 2)(dense_layer_2)
    dense_layer_2 = Dense(128, activation='relu')(dense_layer_2)
    dense_layer_3 = BatchNormalization()(dense_layer_2)
    dense_layer_3 = Dropout(p)(dense_layer_3)
    dense_layer_3 = Dense(10, activation='softmax')(dense_layer_3)
    
    model = Model(inputs = inputs, outputs = dense_layer_3)
    
    return model

# Set level of dropout
p = 0.8

# Set batch size
batch_size = 32

Lets define and compile the model. We run a single epoch on the default learning rate.

In [30]:
bn_model = batch_norm_model(conv_layers, p)

bn_model.compile(optimizer = Adam(lr = 0.001),
                 loss = 'categorical_crossentropy',
                 metrics = ['accuracy'])

bn_model.fit(x = conv_feat,
             y = trn_labels,
             batch_size = batch_size,
             epochs = 1,
             validation_data = (conv_val_feat, val_labels))

Train on 19624 samples, validate on 2800 samples
Epoch 1/1
19624/19624 [==============================] - 11s - loss: 1.4971 - acc: 0.5927 - val_loss: 0.7734 - val_acc: 0.7304


So very fast(!), we see a major improvement over the base VGG16 model. Lets continue running for a few more epochs at a lower learning rate.

In [31]:
bn_model.optimizer.lr = 0.0001

bn_model.fit(x = conv_feat,
             y = trn_labels,
             batch_size = batch_size,
             epochs = 4,
             validation_data = (conv_val_feat, val_labels))

Train on 19624 samples, validate on 2800 samples
Epoch 1/4
19624/19624 [==============================] - 10s - loss: 0.2543 - acc: 0.9223 - val_loss: 0.6655 - val_acc: 0.7568
Epoch 2/4
19624/19624 [==============================] - 10s - loss: 0.1363 - acc: 0.9589 - val_loss: 0.7253 - val_acc: 0.7464
Epoch 3/4
19624/19624 [==============================] - 10s - loss: 0.1001 - acc: 0.9706 - val_loss: 0.6854 - val_acc: 0.7693
Epoch 4/4
19624/19624 [==============================] - 10s - loss: 0.0814 - acc: 0.9752 - val_loss: 0.8364 - val_acc: 0.7364


So we are starting to overfit quite a bit. We have pretty much done as much as we could with the base model, in terms of regularisation. We have added dropout and batchnorm. Time to take a look and see if we can improve the input to the model.

## VGG model with precomputed augmentation and dropout.

We precompute some augmented data in order to reduce the overfitting of out model. We start by setting the level of preprocessing and then define a new bunch of batches.

In [32]:
gen_t = image.ImageDataGenerator(rotation_range = 15,
                                 height_shift_range = 0.05,
                                 shear_range = 0.1,
                                 channel_shift_range = 20,
                                 width_shift_range = 0.1)

da_batches = get_batches(path + 'train',
                         gen_t,
                         batch_size = 44,
                         shuffle = False,
                         target_size = (224, 224))

Found 19624 images belonging to 10 classes.


We then make 5 full cycles (epochs) of the training data, precomputing the convolutional layer on the augmented images.

In [33]:
da_conv_feat = conv_model.predict_generator(da_batches, np.int(da_batches.samples / da_batches.batch_size) * 5)
save_array(path + 'results/conv_computed/da_conv_feat.dat', da_conv_feat)

del da_conv_feat
gc.collect()

0

We can then load the pre-compuited and augmented data.

In [34]:
da_conv_feat = load_array(path + 'results/conv_computed/da_conv_feat.dat')

And then concatenate it with our existing precomputed training data.

In [35]:
da_conv_feat = np.concatenate([da_conv_feat, conv_feat])

We now have 6 times as many features, but luckily order is preserved, so we can just multiply the labels with 6.

In [36]:
da_trn_labels = np.concatenate([trn_labels] * 6)

We can then run the model again, but we try use use bigger dense layers and some more dropout.

In [37]:
def batch_norm_model_bigger(base_model, p):
    
    inputs = Input(shape = base_model[-1].output_shape[1:])
        
    flat_layer = Flatten()(inputs)
        
    dense_layer_1 = Dropout(p)(flat_layer)
    dense_layer_1 = Dense(256, activation='relu')(dense_layer_1)
    dense_layer_2 = BatchNormalization()(dense_layer_1)
    dense_layer_2 = Dropout(p)(dense_layer_2)
    dense_layer_2 = Dense(256, activation='relu')(dense_layer_2)
    dense_layer_3 = BatchNormalization()(dense_layer_2)
    dense_layer_3 = Dropout(p)(dense_layer_3)
    dense_layer_3 = Dense(10, activation='softmax')(dense_layer_3)
    
    model = Model(inputs = inputs, outputs = dense_layer_3)
    
    return model

# Define level of dropout
p = 0.8

# Define batch size
batch_size = 32

Re then compile the model, and run it for a single epoch on the augmented data at the defulat learning rate.

In [38]:
bn_model_bigger = batch_norm_model_bigger(conv_layers, p)

bn_model_bigger.compile(optimizer = Adam(lr = 0.001),
                        loss = 'categorical_crossentropy',
                        metrics = ['accuracy'])

bn_model_bigger.fit(x = da_conv_feat,
                    y = da_trn_labels,
                    batch_size = batch_size,
                    epochs = 1,
                    validation_data = (conv_val_feat, val_labels))

Train on 117744 samples, validate on 2800 samples
Epoch 1/1
117744/117744 [==============================] - 71s - loss: 1.5041 - acc: 0.5137 - val_loss: 0.6635 - val_acc: 0.8154


So this model is a bit heavier. Already we are underfitting quite a bit. We increase the learning rate and run for 4 epochs.

In [39]:
bn_model_bigger.optimizer.lr = 0.01

bn_model_bigger.fit(x = da_conv_feat,
                    y = da_trn_labels,
                    batch_size = batch_size,
                    epochs = 4,
                    validation_data = (conv_val_feat, val_labels))

Train on 117744 samples, validate on 2800 samples
Epoch 1/4
117744/117744 [==============================] - 69s - loss: 0.7201 - acc: 0.7614 - val_loss: 0.5816 - val_acc: 0.8296
Epoch 2/4
117744/117744 [==============================] - 70s - loss: 0.5850 - acc: 0.8104 - val_loss: 0.5832 - val_acc: 0.8296
Epoch 3/4
117744/117744 [==============================] - 69s - loss: 0.5161 - acc: 0.8341 - val_loss: 0.5746 - val_acc: 0.8282
Epoch 4/4
117744/117744 [==============================] - 70s - loss: 0.4650 - acc: 0.8528 - val_loss: 0.5889 - val_acc: 0.8243


Now our training and valuation accuracy seems more ballanced. Let's lower the learning rate again and train for 4 more epochs.

In [40]:
bn_model_bigger.optimizer.lr = 0.0001

bn_model_bigger.fit(x = da_conv_feat,
                    y = da_trn_labels,
                    batch_size = batch_size,
                    epochs = 4,
                    validation_data = (conv_val_feat, val_labels))

Train on 117744 samples, validate on 2800 samples
Epoch 1/4
117744/117744 [==============================] - 69s - loss: 0.4315 - acc: 0.8628 - val_loss: 0.5513 - val_acc: 0.8382
Epoch 2/4
117744/117744 [==============================] - 70s - loss: 0.4094 - acc: 0.8697 - val_loss: 0.5380 - val_acc: 0.8371
Epoch 3/4
117744/117744 [==============================] - 70s - loss: 0.3896 - acc: 0.8774 - val_loss: 0.5447 - val_acc: 0.8311
Epoch 4/4
117744/117744 [==============================] - 68s - loss: 0.3713 - acc: 0.8827 - val_loss: 0.5238 - val_acc: 0.8468


Seems like we are levelling off at this point. Let's attempt a final improvement by using pseudo labelling.

## Pseudolabeling

We're going to try using a combination of [pseudo labeling](http://deeplearning.net/wp-content/uploads/2013/03/pseudo_label_final.pdf) and [knowledge distillation](https://arxiv.org/abs/1503.02531) to allow us to use unlabeled data (i.e. do semi-supervised learning). For our initial experiment we'll use the validation set as the unlabeled data, so that we can see that it is working without using the test set. Afterwards we add the test set as well.

In [41]:
val_pseudo = bn_model_bigger.predict(conv_val_feat, batch_size = 50)

We concatenate thse pseudo labels with our training labels

In [42]:
comb_pseudo = np.concatenate([da_trn_labels, val_pseudo])
comb_feat = np.concatenate([da_conv_feat, conv_val_feat])

And train our model using the extended data set.

In [43]:
bn_model_bigger.optimizer.lr = 0.001

bn_model_bigger.fit(x = comb_feat,
                    y = comb_pseudo,
                    batch_size = batch_size,
                    epochs = 1,
                    validation_data = (conv_val_feat, val_labels))

Train on 120544 samples, validate on 2800 samples
Epoch 1/1
120544/120544 [==============================] - 72s - loss: 0.3758 - acc: 0.8832 - val_loss: 0.5251 - val_acc: 0.8511


We do not really see much of an improvement. Let's try 4 more epochs.

In [44]:
bn_model_bigger.fit(x = comb_feat,
                    y = comb_pseudo,
                    batch_size = batch_size,
                    epochs = 4,
                    validation_data = (conv_val_feat, val_labels))

Train on 120544 samples, validate on 2800 samples
Epoch 1/4
120544/120544 [==============================] - 67s - loss: 0.3585 - acc: 0.8888 - val_loss: 0.5534 - val_acc: 0.8482
Epoch 2/4
120544/120544 [==============================] - 66s - loss: 0.3487 - acc: 0.8919 - val_loss: 0.5538 - val_acc: 0.8571
Epoch 3/4
120544/120544 [==============================] - 67s - loss: 0.3421 - acc: 0.8953 - val_loss: 0.5239 - val_acc: 0.8582
Epoch 4/4
120544/120544 [==============================] - 67s - loss: 0.3359 - acc: 0.8961 - val_loss: 0.5326 - val_acc: 0.8596


Now we are crossing the 0.9 threshold of accuracy. Lets lower the learning rate and train for 4 more epochs and see where that gets us.

In [45]:
bn_model_bigger.optimizer.lr = 0.00001

bn_model_bigger.fit(x = comb_feat,
                    y = comb_pseudo,
                    batch_size = batch_size,
                    epochs = 4,
                    validation_data = (conv_val_feat, val_labels))

Train on 120544 samples, validate on 2800 samples
Epoch 1/4
120544/120544 [==============================] - 67s - loss: 0.3263 - acc: 0.8995 - val_loss: 0.5706 - val_acc: 0.8504
Epoch 2/4
120544/120544 [==============================] - 67s - loss: 0.3211 - acc: 0.9009 - val_loss: 0.5411 - val_acc: 0.8571
Epoch 3/4
120544/120544 [==============================] - 66s - loss: 0.3129 - acc: 0.9039 - val_loss: 0.5434 - val_acc: 0.8554
Epoch 4/4
120544/120544 [==============================] - 66s - loss: 0.3072 - acc: 0.9054 - val_loss: 0.5155 - val_acc: 0.8593


So we do see a pretty nice improvement. Enough to warrent us trying with the entire test set.

In [46]:
test_pseudo = bn_model_bigger.predict(conv_test_feat, batch_size = 2)

We concatenate thse pseudo labels with our training and valuation pseudo labels

In [47]:
comb_pseudo = np.concatenate([comb_pseudo, test_pseudo])
comb_feat = np.concatenate([comb_feat, conv_test_feat])

And train our model using the extended data set.

In [48]:
bn_model_bigger.optimizer.lr = 0.001

bn_model_bigger.fit(x = comb_feat,
                    y = comb_pseudo,
                    batch_size = batch_size,
                    epochs = 1,
                    validation_data = (conv_val_feat, val_labels))

Train on 200270 samples, validate on 2800 samples
Epoch 1/1
200270/200270 [==============================] - 113s - loss: 0.5931 - acc: 0.8343 - val_loss: 0.4971 - val_acc: 0.8543


Hrm, too early to say, but valuation accuracy holds still. Let's run 4 more epochs.

In [49]:
bn_model_bigger.fit(x = comb_feat,
                    y = comb_pseudo,
                    batch_size = batch_size,
                    epochs = 4,
                    validation_data = (conv_val_feat, val_labels))

Train on 200270 samples, validate on 2800 samples
Epoch 1/4
200270/200270 [==============================] - 117s - loss: 0.5552 - acc: 0.8500 - val_loss: 0.4972 - val_acc: 0.8539
Epoch 2/4
200270/200270 [==============================] - 117s - loss: 0.5386 - acc: 0.8556 - val_loss: 0.4828 - val_acc: 0.8575
Epoch 3/4
200270/200270 [==============================] - 117s - loss: 0.5312 - acc: 0.8593 - val_loss: 0.4982 - val_acc: 0.8543
Epoch 4/4
200270/200270 [==============================] - 119s - loss: 0.5230 - acc: 0.8623 - val_loss: 0.4897 - val_acc: 0.8568


Let's lower the learning rate again and run 4 more epochs.

In [50]:
bn_model_bigger.optimizer.lr = 0.00001

bn_model_bigger.fit(x = comb_feat,
                    y = comb_pseudo,
                    batch_size = batch_size,
                    epochs = 4,
                    validation_data = (conv_val_feat, val_labels))

Train on 200270 samples, validate on 2800 samples
Epoch 1/4
200270/200270 [==============================] - 118s - loss: 0.5206 - acc: 0.8629 - val_loss: 0.4906 - val_acc: 0.8539
Epoch 2/4
200270/200270 [==============================] - 117s - loss: 0.5126 - acc: 0.8669 - val_loss: 0.4879 - val_acc: 0.8518
Epoch 3/4
200270/200270 [==============================] - 117s - loss: 0.5100 - acc: 0.8678 - val_loss: 0.4978 - val_acc: 0.8525
Epoch 4/4
200270/200270 [==============================] - 116s - loss: 0.5087 - acc: 0.8691 - val_loss: 0.4902 - val_acc: 0.8554


We are improving slowly. Let's run for 10 more epochs.

In [51]:
bn_model_bigger.fit(x = comb_feat,
                    y = comb_pseudo,
                    batch_size = batch_size,
                    epochs = 10,
                    validation_data = (conv_val_feat, val_labels))

Train on 200270 samples, validate on 2800 samples
Epoch 1/10
200270/200270 [==============================] - 120s - loss: 0.5052 - acc: 0.8692 - val_loss: 0.4937 - val_acc: 0.8521
Epoch 2/10
200270/200270 [==============================] - 118s - loss: 0.5013 - acc: 0.8700 - val_loss: 0.4698 - val_acc: 0.8614
Epoch 3/10
200270/200270 [==============================] - 118s - loss: 0.4982 - acc: 0.8715 - val_loss: 0.4813 - val_acc: 0.8600
Epoch 4/10
200270/200270 [==============================] - 117s - loss: 0.4968 - acc: 0.8718 - val_loss: 0.4782 - val_acc: 0.8611
Epoch 5/10
200270/200270 [==============================] - 118s - loss: 0.4939 - acc: 0.8725 - val_loss: 0.4833 - val_acc: 0.8611
Epoch 6/10
200270/200270 [==============================] - 118s - loss: 0.4917 - acc: 0.8749 - val_loss: 0.4792 - val_acc: 0.8582
Epoch 7/10
200270/200270 [==============================] - 118s - loss: 0.4904 - acc: 0.8746 - val_loss: 0.4837 - val_acc: 0.8543
Epoch 8/10
200270/200270 [=======

Having trained the model, we save the weights.

In [52]:
bn_model_bigger.save_weights('models/batchnorm_vgg16.h5')

## Submitting to Kaggle

We finally submit the improved model to Kaggle.

We start by finding the optimal level of clipping.

In [48]:
valid_batches_pred = get_batches(path + 'valid', batch_size = 50, target_size = (224, 224), shuffle = False, class_mode = None)
conv_val_feat_pred = conv_model.predict_generator(valid_batches_pred, np.int(valid_batches_pred.samples / valid_batches_pred.batch_size))

val_predictions = bn_model_bigger.predict(conv_val_feat_pred, batch_size = 50)

Found 2800 images belonging to 10 classes.


In [57]:
def do_clip(arr, mx): return np.clip(arr, (1 - mx) / 9, mx)

We then proceed to determine the optimal level of clipping using the validation data set.

In [58]:
test_clip = []
for i in np.arange(0.70, 1.0, 0.01):
    test_clip.append([i, categorical_crossentropy(val_labels, do_clip(val_predictions, i)).eval().mean()])

min(test_clip, key = lambda x: x[1])

[1.0000000000000002, 0.48164355679327142]

Here it is said, that no clipping is best. Weird. Lets submit two. One clipped and one not clipped. First we compute the predictions.

In [59]:
test_predictions = bn_model_bigger.predict(conv_test_feat, batch_size = 2)

Define classes

In [61]:
classes = sorted(valid_batches.class_indices, key = valid_batches.class_indices.get)

Then we make a cliping based on earlier experience

In [62]:
sumbit_pred = do_clip(test_predictions, 0.89)

Then we prepare a submission without clipping.

In [65]:
submission_no_clip = pd.DataFrame(test_predictions, columns = classes)
submission_no_clip.insert(0, 'img', [a[8:] for a in test_batches.filenames])
submission_no_clip.head()

,img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,img_81601.jpg,0.046099,0.006723,0.004307,0.000682,0.001594,0.001567,0.026646,0.020867,0.016716,0.874798
1,img_14887.jpg,0.689640,0.007839,0.000621,0.002267,0.001608,0.003189,0.001825,0.000393,0.005292,0.287327
2,img_62885.jpg,0.005301,0.000154,0.000257,0.017661,0.973055,0.000493,0.000514,0.000016,0.000978,0.001571
3,img_45125.jpg,0.002003,0.007311,0.017353,0.000462,0.002328,0.000509,0.656182,0.006239,0.302687,0.004926
4,img_22633.jpg,0.138601,0.055678,0.011123,0.001665,0.004929,0.012565,0.022692,0.007558,0.186375,0.558814


And a submission with clipping

In [66]:
submission_clip = pd.DataFrame(sumbit_pred, columns = classes)
submission_clip.insert(0, 'img', [a[8:] for a in test_batches.filenames])
submission_clip.head()

,img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,img_81601.jpg,0.046099,0.012222,0.012222,0.012222,0.012222,0.012222,0.026646,0.020867,0.016716,0.874798
1,img_14887.jpg,0.689640,0.012222,0.012222,0.012222,0.012222,0.012222,0.012222,0.012222,0.012222,0.287327
2,img_62885.jpg,0.012222,0.012222,0.012222,0.017661,0.890000,0.012222,0.012222,0.012222,0.012222,0.012222
3,img_45125.jpg,0.012222,0.012222,0.017353,0.012222,0.012222,0.012222,0.656182,0.012222,0.302687,0.012222
4,img_22633.jpg,0.138601,0.055678,0.012222,0.012222,0.012222,0.012565,0.022692,0.012222,0.186375,0.558814


Finally we save the two submissions

In [67]:
submission_file_name_no_clip = 'results/augmented-pseudo-vgg-no-clip.gz'
submission_no_clip.to_csv(submission_file_name_no_clip, index = False, compression = 'gzip')

submission_file_name_clip = 'results/augmented-pseudo-vgg-clip.gz'
submission_clip.to_csv(submission_file_name_clip, index = False, compression = 'gzip')

In [70]:
from IPython.display import FileLink
FileLink('results/augmented-pseudo-vgg-no-clip.gz')

/home/ubuntu/kaggle/state-farm-distracted-driver-detection/results/augmented-pseudo-vgg-no-clip.gz

In [71]:
FileLink('results/augmented-pseudo-vgg-clip.gz')

/home/ubuntu/kaggle/state-farm-distracted-driver-detection/results/augmented-pseudo-vgg-clip.gz

Turns out that in this case, the no clipping submission actually performed best, by a absolute 0.03.